**Flujo de agentes implementados**: cadena que simula un proceso de ideación de negocio, donde la IA investiga oportunidades, valida la viabilidad, analiza el mercado y redacta una propuesta final lista para presentar.

### Dependencias

In [1]:
!pip install crewai crewai_tools openai ddgs python-dotenv

nRequirement already satisfied: crewai in /usr/local/lib/python3.12/dist-packages (0.203.1)


### Implementación

#### Setup

Crea tu cuenta en 👉 https://console.groq.com


En el panel, entra a API Keys → Create API Key, copia la clave generada.

In [2]:
import os

os.environ["GROQ_API_KEY"] = ""
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"


Esta clase define una herramienta personalizada para CrewAI que permite a los agentes realizar búsquedas en la web usando DuckDuckGo.

In [3]:
from crewai.tools import BaseTool
from ddgs import DDGS

class MyDuckDuckGoTool(BaseTool):
    name: str = "DuckDuckGo Search Tool"
    description: str = "Search the web using DuckDuckGo"

    def _run(self, query: str = None) -> str:
        try:
            if not query or not isinstance(query, str) or query.strip() == "":
                return "No search query provided."

            ddgs = DDGS()
            results = list(ddgs.text(query, max_results=5))

            if not results:
                return "No results found."

            return "\n".join([r.get("body", "") for r in results if "body" in r])

        except Exception as e:
            return f"DuckDuckGo tool failed: {str(e)}"

Función auxiliar: ejecuta una tarea y, si ocurre un error de límite de peticiones (RateLimitError), espera unos segundos y la vuelve a intentar automáticamente

In [4]:
import time

def run_task_safely(task_func, wait=6):
    """Evita el límite de tasa con pausas automáticas."""
    try:
        return task_func()
    except Exception as e:
        if "RateLimitError" in str(e):
            print(f"Límite alcanzado. Esperando {wait} segundos...")
            time.sleep(wait)
            return task_func()
        else:
            raise e

Define el modelo de lenguaje que usarán los agentes (llm_name) y una herramienta auxiliar (MyDuckDuckGoTool) para hacer búsquedas en la web.

In [5]:
search_tool = MyDuckDuckGoTool()
llm_name = "groq/llama-3.3-70b-versatile"

researcher_llm= "groq/llama-3.3-70b-versatile"
strategist_llm = "groq/llama-3.1-8b-instant"
planner_llm = "groq/llama-3.3-70b-versatile"

#### Comparación con agentes AutoGen

##### Agentes

In [6]:
from crewai import Agent

# =============================================================
# ===============   DEFINICIÓN DE AGENTES CREWAI   =============
# =============================================================

researcher = Agent(
    role="Market Researcher",
    goal="Recolectar información sobre competidores en el mercado de apps de productividad.",
    backstory="Especialista en inteligencia competitiva y tendencias digitales.",
    llm=researcher_llm
)

strategist = Agent(
    role="Strategist",
    goal="Analizar la información de mercado y proponer ventajas competitivas para QuickTask.",
    backstory="Consultor de estrategia digital con experiencia en innovación y diferenciación de productos.",
    llm=strategist_llm
)

planner = Agent(
    role="Planner",
    goal="Convertir las recomendaciones estratégicas en un plan de acción ejecutable y medible.",
    backstory="Gestor de proyectos que traduce la visión estratégica en pasos concretos.",
    llm=planner_llm
)

En LangChain, la definición de agentes suele implicar:



```python
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

llm = OpenAI(model="gpt-4")
tools = [Tool(name="Search", func=search_tool.run, description="Buscar info")]

researcher_chain = initialize_agent(tools, llm, agent_type="zero-shot-react-description")

```



Cada agente en LangChain se crea manualmente y no tiene identidad ni memoria narrativa (role, goal, backstory) de forma nativa.
CrewAI, en cambio, da a cada agente una personalidad, objetivo y rol cooperativo, facilitando tareas colaborativas realistas.

**Beneficio CrewAI**: los agentes no son simples funciones con LLM; tienen propósito, contexto y pueden coordinarse como un equipo.

##### Tasks:

In [7]:
from crewai import Task


# =============================================================
# ===============   DEFINICIÓN DE TAREAS CREWAI   ==============
# =============================================================


task1 = Task(
    description=(
        "Investiga 5 competidores relevantes de aplicaciones de productividad y "
        "resume sus características principales (nombre, función, propuesta de valor)."
    ),
    expected_output="Una lista comparativa con los 5 competidores, sus funciones clave y propuesta de valor.",
    agent=researcher
)

task2 = Task(
    description=(
        "Analiza la lista de competidores recopilada por el investigador y detecta "
        "al menos 3 oportunidades estratégicas claras para diferenciar QuickTask."
    ),
    expected_output="Un análisis estratégico con 3 ventajas competitivas u oportunidades de mejora.",
    agent=strategist,
    depends_on=[task1]
)

task3 = Task(
    description=(
        "Diseña un plan de acción de 5 pasos para implementar una de las oportunidades detectadas "
        "por el estratega. El plan debe incluir fases, responsables y métricas de éxito."
    ),
    expected_output="Un plan de acción ejecutable y claro con pasos y métricas de implementación.",
    agent=planner,
    depends_on=[task2]
)


En LangChain, las dependencias entre pasos deben manejarse manualmente:

**Beneficio CrewAI**: las dependencias (depends_on) se manejan automáticamente.
Cada agente sabe cuándo actuar y con qué datos, sin necesidad de secuencias manuales

##### Crew

In [8]:
from crewai import Crew

# =============================================================
# ===============   CREACIÓN Y EJECUCIÓN DE CREW   =============
# =============================================================

crew = Crew(
    agents=[researcher, strategist, planner],
    tasks=[task1, task2, task3],
    verbose=True,
    process='sequential'  # ejecución estricta y ordenada
)

result = run_task_safely(lambda: crew.kickoff())

print("\n FINAL RESULT:\n")
print(result)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4e160c09-42ab-4f73-b0a4-6cf8eb132fc3                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Researcher                                                                                       │
│                                                                                                                 │
│  Task: Investiga 5 competidores relevantes de aplicaciones de productividad y resume sus características        │
│  principales (nombre, función, propuesta de valor).                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Researcher                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  A continuación, se presenta una lista comparativa con 5 competidores relevantes de aplicaciones de             │
│  productividad, junto con sus características principales:                                                      │
│                                                                                                                 │
│  1. **Trello**:                                                                                                 │
│     - Nombre: Trello                                                                                            │
│     - Función: Gestión de proyectos y tareas a través de tableros visuales y tarjetas.                          │
│     - Propuesta de valor: Ofrece una forma visual y flexible de organizar y gestionar proyectos, permitiendo a  │
│  los usuarios crear tableros personalizados con diferentes listas y tarjetas para seguir el progreso de sus     │
│  tareas.                                                                                                        │
│                                                                                                                 │
│  2. **Evernote**:                                                                                               │
│     - Nombre: Evernote                                                                                          │
│     - Función: Toma de notas y organización de información personal y profesional.                              │
│     - Propuesta de valor: Permite a los usuarios capturar, organizar y almacenar información de diversas        │
│  fuentes en notas, permitiendo la búsqueda y sincronización en múltiples dispositivos, lo que facilita la       │
│  productividad y el acceso a la información en cualquier momento.                                               │
│                                                                                                                 │
│  3. **Asana**:                                                                                                  │
│     - Nombre: Asana                                                                                             │
│     - Función: Gestión de tareas y proyectos para equipos y individuos.                                         │
│     - Propuesta de valor: Ayuda a los usuarios a planificar, organizar y realizar un seguimiento de sus tareas  │
│  y proyectos, mejorando la colaboración y la productividad del equipo a través de asignaciones de tareas,       │
│  plazos y seguimiento del progreso.                                                                             │
│                                                                                                                 │
│  4. **Notion**:                                                                                                 │
│     - Nombre: Notion                                                                                            │
│     - Función: Aplicación de productividad y nota que combina la toma de notas, la gestión de proyectos y la    │
│  base de datos.                                                                                                 │
│     - Propuesta de valor: Ofrece una plataforma versátil para tomar notas, gestionar proyectos, crear bases de  │
│  datos y colaborar con otros, todo en un solo lugar, lo

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: bbd8505c-aac5-4c68-bbf5-96ff549a1d4e                                                                     │
│  Agent: Market Researcher                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Planner                                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Plan de Acción para Implementar la Ventaja Competitiva de la Integración de Inteligencia Artificial en       │
│  QuickTask**                                                                                                    │
│                                                                                                                 │
│  **Fase 1: Análisis y Planificación (Semana 1-4)**                                                              │
│                                                                                                                 │
│  * Responsable: Equipo de Desarrollo de QuickTask                                                               │
│  * Objetivos:                                                                                                   │
│   + Identificar las características y algoritmos de IA que se pueden integrar en QuickTask                      │
│   + Diseñar la arquitectura de la plataforma para incorporar la IA                                              │
│   + Establecer los parámetros y métricas para medir el éxito de la implementación                               │
│  * Métricas de éxito:                                                                                           │
│   + Tiempo de desarrollo y prueba de la funcionalidad de IA                                                     │
│   + Número de usuarios que utilizan la funcionalidad de IA                                                      │
│   + Retroalimentación y satisfacción de los usuarios con la funcionalidad de IA                                 │
│                                                                                                                 │
│  **Fase 2: Desarrollo y Prueba (Semana 5-12)**                                                                  │
│                                                                                                                 │
│  * Responsable: Equipo de Desarrollo de QuickTask                                                               │
│  * Objetivos:                                                                                                   │
│   + Desarrollar y probar la funcionalidad de IA para la automatización de tareas y recomendaciones              │
│  personalizadas                                                                                                 │
│   + Integrar la funcionalidad de IA con la plataforma existente de QuickTask                                    │
│   + Realizar pruebas de rendimiento y seguridad de la funcionalidad de IA                                       │
│  * Métricas de éxito:                                                                                           │
│   + Funcionalidad de IA implementada y probada con éxito                                                        │
│   + Tiempo de respuesta y rendimiento de la funcionalidad de IA                                                 │
│   + Número de errores y problemas identificados y resueltos durante la prueba                                   │
│                                                                                                                 │
│  **Fase 3: Implementación y Lanzamiento (Semana 13-16)**                                                        │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 767a7256-6547-42e8-a1fb-66c1282f636a                                                                     │
│  Agent: Planner                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4e160c09-42ab-4f73-b0a4-6cf8eb132fc3                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Plan de Acción para Implementar la Ventaja Competitiva de la Integración de Inteligencia       │
│  Artificial en QuickTask**                                                                                      │
│                                                                                                                 │
│  **Fase 1: Análisis y Planificación (Semana 1-4)**                                                              │
│                                                                                                                 │
│  * Responsable: Equipo de Desarrollo de QuickTask                                                               │
│  * Objetivos:                                                                                                   │
│   + Identificar las características y algoritmos de IA que se pueden integrar en QuickTask                      │
│   + Diseñar la arquitectura de la plataforma para incorporar la IA                                              │
│   + Establecer los parámetros y métricas para medir el éxito de la implementación                               │
│  * Métricas de éxito:                                                                                           │
│   + Tiempo de desarrollo y prueba de la funcionalidad de IA                                                     │
│   + Número de usuarios que utilizan la funcionalidad de IA                                                      │
│   + Retroalimentación y satisfacción de los usuarios con la funcionalidad de IA                                 │
│                                                                                                                 │
│  **Fase 2: Desarrollo y Prueba (Semana 5-12)**                                                                  │
│                                                                                                                 │
│  * Responsable: Equipo de Desarrollo de QuickTask                                                               │
│  * Objetivos:                                                                                                   │
│   + Desarrollar y probar la funcionalidad de IA para la automatización de tareas y recomendaciones              │
│  personalizadas                                                                                                 │
│   + Integrar la funcionalidad de IA con la plataforma existente de QuickTask                                    │
│   + Realizar pruebas de rendimiento y seguridad de la funcionalidad de IA                                       │
│  * Métricas de éxito:                                                                                           │
│   + Funcionalidad de IA implementada y probada con éxito                                                        │
│   + Tiempo de respuesta y rendimiento de la funcionalidad de IA                                                 │
│   + Número de errores y problemas identificados y resueltos durante la prueba                                   │
│                                                                                                                 │
│  **Fase 3: Implementación y Lanzamiento (Semana 13-16)


 FINAL RESULT:

**Plan de Acción para Implementar la Ventaja Competitiva de la Integración de Inteligencia Artificial en QuickTask**

**Fase 1: Análisis y Planificación (Semana 1-4)**

* Responsable: Equipo de Desarrollo de QuickTask
* Objetivos:
 + Identificar las características y algoritmos de IA que se pueden integrar en QuickTask
 + Diseñar la arquitectura de la plataforma para incorporar la IA
 + Establecer los parámetros y métricas para medir el éxito de la implementación
* Métricas de éxito:
 + Tiempo de desarrollo y prueba de la funcionalidad de IA
 + Número de usuarios que utilizan la funcionalidad de IA
 + Retroalimentación y satisfacción de los usuarios con la funcionalidad de IA

**Fase 2: Desarrollo y Prueba (Semana 5-12)**

* Responsable: Equipo de Desarrollo de QuickTask
* Objetivos:
 + Desarrollar y probar la funcionalidad de IA para la automatización de tareas y recomendaciones personalizadas
 + Integrar la funcionalidad de IA con la plataforma existente de QuickTask